In [26]:
import os
from music21 import converter, pitch, interval, instrument, note, chord
import numpy as np
import json

data = './data/'

if not os.path.exists(data):
    os.makedirs(data)

In [27]:
songList = [song for song in os.listdir(data) if song.lower().find('giuliani')>-1]
print(len(songList) , 'songs found by Giuliani')

88 songs found by Giuliani


In [28]:
def monophonic(stream):
    return len(instrument.partitionByInstrument(stream).parts) == 1

songList = [song for song in songList if monophonic(converter.parse(data+song))]
print(len(songList) , 'monophonic songs found by Giuliani')

32 monophonic songs found by Giuliani


In [29]:
if not os.path.exists('./giuliani-monophonic/'):
    os.makedirs('./giuliani-monophonic/')

for song in songList:
    os.system('cp ' + data + song + ' ./giuliani-monophonic/' + song)

In [30]:
streams = [converter.parse('./giuliani-monophonic/' + song) for song in songList]
chordifiedStreams = [stream.chordify() for stream in streams]

In [35]:
originalChords = [[] for _ in chordifiedStreams]
originalDurations = [[] for _ in chordifiedStreams]
originalKeys = []

for i, song in enumerate(chordifiedStreams):
    originalKeys.append(str(song.analyze('key')))

    for measure in song.recurse().getElementsByClass('Measure'):
        for element in measure:
            if isinstance(element, chord.Chord):
                originalChords[i].append('.'.join(str(n) for n in element.pitches))
                originalDurations[i].append(element.duration.quarterLength)

print('Original chords: ', originalChords[0][:10])

Original chords:  ['D5', 'G3.D5', 'B3.D5', 'D4.D5', 'G3.B4', 'B3.B4', 'D4.B4', 'G3.G4', 'B3.G4', 'D4.G4']


In [36]:
print(originalKeys)

mostCommonKey = max(set(originalKeys), key=originalKeys.count)

print(mostCommonKey , 'is the most common key with' , originalKeys.count(mostCommonKey) , 'occurrences')

['G major', 'C major', 'G major', 'a minor', 'C major', 'a minor', 'd minor', 'D major', 'C major', 'G major', 'G major', 'C major', 'a minor', 'A major', 'A major', 'C major', 'D major', 'C major', 'D major', 'D major', 'C major', 'C major', 'A major', 'a minor', 'G major', 'A major', 'a minor', 'G major', 'C major', 'C major', 'a minor', 'g minor']
C major is the most common key with 10 occurrences


In [37]:
mckChords = []
mckDurations = []

for key in originalKeys:
    if key == mostCommonKey:
        mckChords.append(originalChords[originalKeys.index(key)])
        mckDurations.append(originalDurations[originalKeys.index(key)])

print(len(mckChords) , 'songs in the most common key')

10 songs in the most common key


In [41]:
uniqueChords = np.unique([i for s in originalChords for i in s])
chordToInt = dict(zip(uniqueChords, list(range(0, len(uniqueChords)))))

uniqueDurations = np.unique([i for s in originalDurations for i in s])
durationToInt = dict(zip(uniqueDurations, list(range(0, len(uniqueDurations)))))

intToChord = {i: c for c, i in chordToInt.items()}
intToDuration = {i: c for c, i in durationToInt.items()}

In [43]:
trainingWidth = 32

trainChords = []
trainDurations = []

for i in range(len(mckChords)):
    chordList = [chordToInt[chord] for chord in mckChords[i]]
    durationList = [durationToInt[duration] for duration in mckDurations[i]]

    for j in range(len(chordList) - trainingWidth):
        trainChords.append(chordList[j:j+trainingWidth])
        trainDurations.append(durationList[j:j+trainingWidth])